In [240]:
import pdfplumber, re, pandas as pd, os

# Extract

In [ ]:
def extractStatement(path: str, possible_date_formats: str, password: str = None, return_non_trans_sentences: bool = False) -> list[str]:
    # Returnes a \n-seperated string with all the date-like lines from a pdf
    with pdfplumber.open(path, password=password) as file:
        pages = file.pages # Get all pages from PDF
        print(f'Reading pdf from "{path}".\n{len(pages)} pages in pdf')
        
        words = [page.extract_text() for page in pages] # Extract all words from each page
        words = '\n\n'.join(words) # Convert list of strings from each page to a single string
        sentences = words.splitlines() # Convert string to list of sentences
        
        trans_sentences = [] # Saves transaction-like sentences
        non_trans_sentences = [] # Saves non-transaction-like sentences
        
        for sentence in sentences:
            # Check all possible date regexes
            date_checks = [bool(re.search(possible_date_format, sentence)) for possible_date_format in possible_date_formats]
            is_date = any(date_checks)
            # print(is_date)
            
            if is_date: # If date-like string found
                trans_sentences.append(sentence)
            
            else: # If date-like string not found
                non_trans_sentences.append(sentence)
        
        print(f'{len(trans_sentences)} date-like sentence found (out of {len(sentences)})\n')
        
        if return_non_trans_sentences:
            return trans_sentences, non_trans_sentences # Return sentences containing date-like values
        else:
            return trans_sentences

# Read Data

In [244]:
def extractDateFromString(string: str):
    # Search for date basis `possible_date_formats` declared earlier
    # If match is found w regex at 0, result will be retured and no further search will be done
    for regex in possible_date_formats:
        result = re.search(regex, string)
        if bool(result):
            return result.group()
    return -1

In [245]:
def extractAmountFromString(string: str):
    # Splits basis ' '
    # Reverses the list and which even element can be type casted to float will be returned
    values = string.split(' ')
    values.reverse()
    for value in values:
        value = value.replace(',', '')
        try:
            return str(float(value)) # Typecasted to str to avoid e-notation
        except ValueError as v:
            continue

In [246]:
def extractCommentsFromString(string: str):
    result = re.findall('[a-zA-Z]', string)
    result = ''.join(result)
    return result

In [249]:
def transformStatementV2(transations: list[str]) -> pd.DataFrame:
    df = pd.DataFrame(transations, columns=['raw_data'])
    
    # Normalizing the string-like transation data
    df['raw_data'] = df['raw_data'].astype(str).str.strip().str.lower()
    
    # Identifying debit/credit transations
    df['is_credit'] = 0
    df.loc[(df['raw_data'].astype(str).str.endswith('cr')), 'is_credit'] = 1
    
    # Removing 'cr' from raw_data for credit transations
    df.loc[(df['is_credit'] == 1), 'raw_data'] = df['raw_data'].str[:-2]
    
    # Extracting date from raw_data
    df['date'] = df['raw_data'].apply(lambda x: extractDateFromString(x))
    # df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y') # TODO: Handle additional formats
    
    # Extracting amount from raw_data
    df['amount'] = df['raw_data'].apply(lambda x: extractAmountFromString(x))
    
    # Extracting comments from raw_data
    df['comments'] = df['raw_data'].astype(str).str.replace('[^a-zA-Z]', ' ', regex=True, case=False)
    
    return df

# Implementaion

In [251]:
possible_date_formats = [r'([0-9]{2}/[0-9]{2}/[0-9]{4})', r'([0-9]{2}-[a-zA-Z]{3}-[0-9]{2})'] # CONFIG: Pass additional regexes as you deem fit per your data

In [254]:
def etl(path: str, password: str, regexes: list):
    e = extractStatement(path=path, password=password, possible_date_formats=regexes)
    t = transformStatementV2(e)
    return t

In [ ]:
# folder_path = r'D:\work\code_and_stuff\Bank Statement Reader\PDFs\HDFC'
# file_paths = [folder_path + '\\' + file for file in os.listdir(folder_path)]
# dfs = [etl(file_path, '<PUT PASSWORD HERE>', possible_date_formats) for file_path in file_paths]

file_paths = [r'D:\work\code_and_stuff\Bank Statement Reader\PDFs\ICICI\2024-23-11--19-13-26-creditAnnualStmt.pdf']
dfs = [etl(file_path, None, possible_date_formats) for file_path in file_paths]

print(f'File read: {len(dfs)}')

df = pd.concat(dfs, axis=0, ignore_index=True)
print(df.shape)

df['bank'] = 'ICICI'

df.to_csv(r'D:\work\code_and_stuff\Bank Statement Reader\Processed\ICICI.csv', index=False)

Reading pdf from "D:\work\code_and_stuff\Bank Statement Reader\PDFs\ICICI\2024-23-11--19-13-26-creditAnnualStmt.pdf".
15 pages in pdf
353 date-like sentence found (out of 616)

File read: 1
(353, 5)
